In [1]:
import keras
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=12)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=15)    # legend fontsize

In [2]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b:
            ch = 1.0
            stop = True
        elif x[0]>= b and x[1]>=b:
            rt = 0
            x = np.array([0.0, 0.0])
    return rt+ndt, ch

In [3]:
data_df = pd.read_csv('../../_00_data/Sepulveda2020_perceptual_prep.csv', index_col=0)

min_rt = data_df.groupby(by='sbj')['rt'].min()

In [4]:
tr_df, ts_df = data_df.copy(), data_df.copy()

obs_train = np.asarray(data_df[['rt', 'choice', 'sbj']])
X_train = np.asarray(data_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1', 'goal']])

obs_test = np.asarray(data_df[['rt', 'choice', 'sbj']])
X_test = np.asarray(data_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1', 'goal']])

print(X_train.shape, X_test.shape)

(7555, 5) (7555, 5)


In [5]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.1**10

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):

    sbj_indices = tf.cast(data[:, 2], tf.int32)

    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))

    ll1 = (1 - data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 1]))
    ll2 = (data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 0]))

    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2, 1e-14)), axis=-1)

In [6]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=2, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [7]:
layers_neuron = [10, 10, 10, 8]
input_shape = 5
output_shape = 2
activation = 'tanh'
n_sbj = tr_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                60        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 8)                 88        
                                                                 
 out_layer (Out_layer)       (None, 2)                 114       
                                                                 
Total params: 482 (1.88 KB)
Trainable params: 482 (1.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
sbj_indices = tf.cast(obs_train[:, 2], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [9]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=3000)

Epoch 1/3000
1/1 [==============================] - 1s 632ms/step - loss: 72439.1719
Epoch 2/3000
1/1 [==============================] - 0s 3ms/step - loss: 71876.1641
Epoch 3/3000
1/1 [==============================] - 0s 3ms/step - loss: 71475.5469
Epoch 4/3000
1/1 [==============================] - 0s 5ms/step - loss: 71141.2500
Epoch 5/3000
1/1 [==============================] - 0s 5ms/step - loss: 70844.7500
Epoch 6/3000
1/1 [==============================] - 0s 4ms/step - loss: 70573.4219
Epoch 7/3000
1/1 [==============================] - 0s 4ms/step - loss: 70320.3672
Epoch 8/3000
1/1 [==============================] - 0s 4ms/step - loss: 70081.3906
Epoch 9/3000
1/1 [==============================] - 0s 4ms/step - loss: 69853.6562
Epoch 10/3000
1/1 [==============================] - 0s 3ms/step - loss: 69635.1250
Epoch 11/3000
1/1 [==============================] - 0s 3ms/step - loss: 69424.2188
Epoch 12/3000
1/1 [==============================] - 0s 4ms/step - loss: 69219.7188

1/1 [==============================] - 0s 4ms/step - loss: 59163.2812
Epoch 99/3000
1/1 [==============================] - 0s 5ms/step - loss: 59092.6328
Epoch 100/3000
1/1 [==============================] - 0s 4ms/step - loss: 59022.7031
Epoch 101/3000
1/1 [==============================] - 0s 4ms/step - loss: 58953.4922
Epoch 102/3000
1/1 [==============================] - 0s 4ms/step - loss: 58884.9766
Epoch 103/3000
1/1 [==============================] - 0s 4ms/step - loss: 58817.1562
Epoch 104/3000
1/1 [==============================] - 0s 4ms/step - loss: 58750.0195
Epoch 105/3000
1/1 [==============================] - 0s 4ms/step - loss: 58683.5586
Epoch 106/3000
1/1 [==============================] - 0s 4ms/step - loss: 58617.7578
Epoch 107/3000
1/1 [==============================] - 0s 4ms/step - loss: 58552.6094
Epoch 108/3000
1/1 [==============================] - 0s 4ms/step - loss: 58488.1016
Epoch 109/3000
1/1 [==============================] - 0s 4ms/step - loss: 58424.2

1/1 [==============================] - 0s 4ms/step - loss: 54560.0703
Epoch 195/3000
1/1 [==============================] - 0s 4ms/step - loss: 54526.6992
Epoch 196/3000
1/1 [==============================] - 0s 4ms/step - loss: 54493.5078
Epoch 197/3000
1/1 [==============================] - 0s 4ms/step - loss: 54460.5000
Epoch 198/3000
1/1 [==============================] - 0s 4ms/step - loss: 54427.6523
Epoch 199/3000
1/1 [==============================] - 0s 4ms/step - loss: 54394.9961
Epoch 200/3000
1/1 [==============================] - 0s 4ms/step - loss: 54362.4961
Epoch 201/3000
1/1 [==============================] - 0s 4ms/step - loss: 54330.1875
Epoch 202/3000
1/1 [==============================] - 0s 4ms/step - loss: 54298.0234
Epoch 203/3000
1/1 [==============================] - 0s 4ms/step - loss: 54266.0312
Epoch 204/3000
1/1 [==============================] - 0s 4ms/step - loss: 54234.1953
Epoch 205/3000
1/1 [==============================] - 0s 4ms/step - loss: 54202.

1/1 [==============================] - 0s 4ms/step - loss: 51935.6797
Epoch 291/3000
1/1 [==============================] - 0s 4ms/step - loss: 51912.6406
Epoch 292/3000
1/1 [==============================] - 0s 4ms/step - loss: 51889.6602
Epoch 293/3000
1/1 [==============================] - 0s 4ms/step - loss: 51866.7383
Epoch 294/3000
1/1 [==============================] - 0s 4ms/step - loss: 51843.8789
Epoch 295/3000
1/1 [==============================] - 0s 3ms/step - loss: 51821.0703
Epoch 296/3000
1/1 [==============================] - 0s 4ms/step - loss: 51798.3320
Epoch 297/3000
1/1 [==============================] - 0s 4ms/step - loss: 51775.6484
Epoch 298/3000
1/1 [==============================] - 0s 3ms/step - loss: 51753.0234
Epoch 299/3000
1/1 [==============================] - 0s 4ms/step - loss: 51730.4531
Epoch 300/3000
1/1 [==============================] - 0s 3ms/step - loss: 51707.9453
Epoch 301/3000
1/1 [==============================] - 0s 4ms/step - loss: 51685.

1/1 [==============================] - 0s 4ms/step - loss: 49940.6016
Epoch 387/3000
1/1 [==============================] - 0s 4ms/step - loss: 49921.6250
Epoch 388/3000
1/1 [==============================] - 0s 4ms/step - loss: 49902.6797
Epoch 389/3000
1/1 [==============================] - 0s 4ms/step - loss: 49883.7695
Epoch 390/3000
1/1 [==============================] - 0s 4ms/step - loss: 49864.8828
Epoch 391/3000
1/1 [==============================] - 0s 4ms/step - loss: 49846.0312
Epoch 392/3000
1/1 [==============================] - 0s 4ms/step - loss: 49827.1953
Epoch 393/3000
1/1 [==============================] - 0s 4ms/step - loss: 49808.3984
Epoch 394/3000
1/1 [==============================] - 0s 4ms/step - loss: 49789.6328
Epoch 395/3000
1/1 [==============================] - 0s 4ms/step - loss: 49770.8906
Epoch 396/3000
1/1 [==============================] - 0s 3ms/step - loss: 49752.1797
Epoch 397/3000
1/1 [==============================] - 0s 4ms/step - loss: 49733.

...

1/1 [==============================] - 0s 4ms/step - loss: 33021.6562
Epoch 2578/3000
1/1 [==============================] - 0s 4ms/step - loss: 33024.2891
Epoch 2579/3000
1/1 [==============================] - 0s 5ms/step - loss: 33030.6562
Epoch 2580/3000
1/1 [==============================] - 0s 5ms/step - loss: 32981.8828
Epoch 2581/3000
1/1 [==============================] - 0s 5ms/step - loss: 32986.5703
Epoch 2582/3000
1/1 [==============================] - 0s 5ms/step - loss: 32976.3516
Epoch 2583/3000
1/1 [==============================] - 0s 5ms/step - loss: 32983.7500
Epoch 2584/3000
1/1 [==============================] - 0s 5ms/step - loss: 32974.7500
Epoch 2585/3000
1/1 [==============================] - 0s 4ms/step - loss: 32986.0547
Epoch 2586/3000
1/1 [==============================] - 0s 5ms/step - loss: 32978.2539
Epoch 2587/3000
1/1 [==============================] - 0s 5ms/step - loss: 32989.6953
Epoch 2588/3000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 3ms/step - loss: 32893.6211
Epoch 2673/3000
1/1 [==============================] - 0s 3ms/step - loss: 32904.7656
Epoch 2674/3000
1/1 [==============================] - 0s 3ms/step - loss: 32883.8281
Epoch 2675/3000
1/1 [==============================] - 0s 3ms/step - loss: 32894.5078
Epoch 2676/3000
1/1 [==============================] - 0s 3ms/step - loss: 32875.9531
Epoch 2677/3000
1/1 [==============================] - 0s 3ms/step - loss: 32889.0547
Epoch 2678/3000
1/1 [==============================] - 0s 3ms/step - loss: 32873.7266
Epoch 2679/3000
1/1 [==============================] - 0s 3ms/step - loss: 32889.1055
Epoch 2680/3000
1/1 [==============================] - 0s 3ms/step - loss: 32875.5859
Epoch 2681/3000
1/1 [==============================] - 0s 3ms/step - loss: 32891.1523
Epoch 2682/3000
1/1 [==============================] - 0s 3ms/step - loss: 32876.8945
Epoch 2683/3000
1/1 [==============================] - 0s 3ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 32780.4453
Epoch 2768/3000
1/1 [==============================] - 0s 3ms/step - loss: 32778.8203
Epoch 2769/3000
1/1 [==============================] - 0s 3ms/step - loss: 32797.7500
Epoch 2770/3000
1/1 [==============================] - 0s 3ms/step - loss: 32764.2090
Epoch 2771/3000
1/1 [==============================] - 0s 3ms/step - loss: 32793.6953
Epoch 2772/3000
1/1 [==============================] - 0s 3ms/step - loss: 32805.8359
Epoch 2773/3000
1/1 [==============================] - 0s 3ms/step - loss: 32874.5469
Epoch 2774/3000
1/1 [==============================] - 0s 3ms/step - loss: 32815.3594
Epoch 2775/3000
1/1 [==============================] - 0s 3ms/step - loss: 32867.6602
Epoch 2776/3000
1/1 [==============================] - 0s 3ms/step - loss: 32795.0898
Epoch 2777/3000
1/1 [==============================] - 0s 3ms/step - loss: 32792.0664
Epoch 2778/3000
1/1 [==============================] - 0s 3ms/step - l

1/1 [==============================] - 0s 3ms/step - loss: 32710.7070
Epoch 2863/3000
1/1 [==============================] - 0s 4ms/step - loss: 32719.6309
Epoch 2864/3000
1/1 [==============================] - 0s 3ms/step - loss: 32708.8418
Epoch 2865/3000
1/1 [==============================] - 0s 4ms/step - loss: 32717.7695
Epoch 2866/3000
1/1 [==============================] - 0s 3ms/step - loss: 32707.0137
Epoch 2867/3000
1/1 [==============================] - 0s 4ms/step - loss: 32715.9160
Epoch 2868/3000
1/1 [==============================] - 0s 4ms/step - loss: 32705.2148
Epoch 2869/3000
1/1 [==============================] - 0s 4ms/step - loss: 32714.0430
Epoch 2870/3000
1/1 [==============================] - 0s 3ms/step - loss: 32703.4863
Epoch 2871/3000
1/1 [==============================] - 0s 3ms/step - loss: 32712.9160
Epoch 2872/3000
1/1 [==============================] - 0s 4ms/step - loss: 32702.6367
Epoch 2873/3000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 3ms/step - loss: 32642.4844
Epoch 2958/3000
1/1 [==============================] - 0s 3ms/step - loss: 32632.2598
Epoch 2959/3000
1/1 [==============================] - 0s 3ms/step - loss: 32641.0898
Epoch 2960/3000
1/1 [==============================] - 0s 3ms/step - loss: 32630.9746
Epoch 2961/3000
1/1 [==============================] - 0s 3ms/step - loss: 32639.9043
Epoch 2962/3000
1/1 [==============================] - 0s 3ms/step - loss: 32629.6797
Epoch 2963/3000
1/1 [==============================] - 0s 3ms/step - loss: 32638.6543
Epoch 2964/3000
1/1 [==============================] - 0s 3ms/step - loss: 32628.1719
Epoch 2965/3000
1/1 [==============================] - 0s 3ms/step - loss: 32636.6289
Epoch 2966/3000
1/1 [==============================] - 0s 3ms/step - loss: 32625.9434
Epoch 2967/3000
1/1 [==============================] - 0s 3ms/step - loss: 32634.1719
Epoch 2968/3000
1/1 [==============================] - 0s 3ms/step - l

In [10]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

np.save('input_perceptual.npy', X_test)
np.save('output_perceptual.npy', pred_v)

237/237 [==============================] - 0s 330us/step


In [11]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'goal':[],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['goal'].append(ts_df.goal[s])
        pred_bhv['value_diff'].append(ts_df['Right Value - Left Value'][s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - L)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)
pred_bhv.to_csv('Sp2020_preceptual_complete.csv')